In [ ]:
import pandas as pd
import folium
import numpy as np

In [ ]:
ori_gnss_pd = pd.read_csv('/workspaces/ublox-gnss-eval-client/temp/notebooks/f9k_sync1.csv')

In [ ]:
ori_gnss_pd.head()

In [ ]:
ori_gnss_np = ori_gnss_pd[['Latitude', 'Longitude']].to_numpy()

In [ ]:
# Define meters-to-degree conversion for latitude and longitude
lat_per_meter = 1 / 111320.0   # degrees per meter for latitude
mean_lat = np.mean(ori_gnss_np[:, 0])
lon_per_meter = 1 / (111320 * np.cos(np.deg2rad(mean_lat)))  # degrees per meter for longitude

error_m = 0.05  # 5 cm

# Random errors in degrees:
lat_errors = np.random.uniform(-error_m*lat_per_meter, error_m*lat_per_meter, size=ori_gnss_np.shape[0])
lon_errors = np.random.uniform(-error_m*lon_per_meter, error_m*lon_per_meter, size=ori_gnss_np.shape[0])
random_errors = np.stack([lat_errors, lon_errors], axis=1)

# Add errors to the original trace
new_gnss_np = ori_gnss_np + random_errors

In [ ]:

# Calculate map center (mean of first trace, or combine for both)
center = [ori_gnss_np[:, 0].mean(), ori_gnss_np[:, 1].mean()]

# Create Folium map
m = folium.Map(location=center, zoom_start=17)

# Trace 1: Blue
coordinates1 = list(map(tuple, ori_gnss_np))
folium.PolyLine(coordinates1, color='blue', weight=3, opacity=0.8, tooltip='Trace 1').add_to(m)
folium.Marker(coordinates1[0], popup='Start 1', icon=folium.Icon(color='green')).add_to(m)
folium.Marker(coordinates1[-1], popup='End 1', icon=folium.Icon(color='red')).add_to(m)

# Trace 2: Orange
coordinates2 = list(map(tuple, new_gnss_np))
folium.PolyLine(coordinates2, color='orange', weight=3, opacity=0.8, tooltip='Trace 2').add_to(m)
folium.Marker(coordinates2[0], popup='Start 2', icon=folium.Icon(color='purple')).add_to(m)
folium.Marker(coordinates2[-1], popup='End 2', icon=folium.Icon(color='darkred')).add_to(m)

m

In [ ]:
# dtw compare

from dtw import dtw
from haversine import haversine


# Haversine distance function for DTW
def gnss_distance(x, y):
    """Calculate distance between two (lat, lon) points in meters."""
    return haversine((x[0], x[1]), (y[0], y[1]), unit='m')

alignment = dtw(
    ori_gnss_np, 
    new_gnss_np,
    dist_method =gnss_distance,
    keep_internals=True)

dtw_distance = alignment.distance
normalized_distance = alignment.normalizedDistance
print(f"DTW distance: {dtw_distance:.2f} m")
print(f"Normalized DTW distance: {normalized_distance:.2f} m")